In [4]:
# Load FAISS artifacts, perform vector search for a user query, and answer with OpenAI Chat
# !pip install -q numpy faiss-cpu openai==1.*

import os
import json
import numpy as np
import faiss
from openai import OpenAI

# Config
CACHE_DIR = "rag_cache"
INDEX_PATH = os.path.join(CACHE_DIR, "faiss.index")
IDMAP_PATH = os.path.join(CACHE_DIR, "id_map.npy")
TEXTS_PATH = os.path.join(CACHE_DIR, "rag_texts.jsonl")
MANIFEST_PATH = os.path.join(CACHE_DIR, "manifest.json")
EMBED_MODEL = "text-embedding-3-small"
LLM_MODEL = "gpt-4o-mini"
TOP_K = 5

if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("Set OPENAI_API_KEY in your environment before running this cell.")

# 1) Load artifacts
if not (os.path.exists(INDEX_PATH) and os.path.exists(IDMAP_PATH) and os.path.exists(TEXTS_PATH)):
    raise RuntimeError("Artifacts not found. Run the embedding/build step first (see Untitled-2.ipynb).")

index = faiss.read_index(INDEX_PATH)
id_map = np.load(IDMAP_PATH)
texts = {}
with open(TEXTS_PATH, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        texts[int(obj["id"])] = obj["rag_text"]

# 2) Helper functions
client = OpenAI()

def embed_query(query: str) -> np.ndarray:
    resp = client.embeddings.create(model=EMBED_MODEL, input=[query])
    q = np.array(resp.data[0].embedding, dtype=np.float32)
    q = q / np.linalg.norm(q)
    return q

def search(q_vec: np.ndarray, k: int = TOP_K):
    scores, idxs = index.search(q_vec.reshape(1, -1), k)
    return scores[0], id_map[idxs[0]]

def format_context(hit_ids, max_chars_per_chunk: int = 2000) -> str:
    blocks = []
    for hid in hit_ids:
        txt = texts.get(int(hid), "")
        if max_chars_per_chunk and len(txt) > max_chars_per_chunk:
            txt = txt[:max_chars_per_chunk] + "\n..."
        blocks.append(f"[id={hid}]\n{txt}")
    return "\n\n---\n\n".join(blocks)

def answer_with_rag(query: str, k: int = TOP_K, temperature: float = 0.2) -> str:
    q = embed_query(query)
    scores, hit_ids = search(q, k)
    context = format_context(hit_ids.tolist())

    system_prompt = (
        "You are a helpful assistant. That helps to navigate throughout the platform called uyren.ai. The is an online learning platform. With a bunch of courses "
        "You answer the requests based on the data you get."
    )
    user_prompt = f"Question:\n{query}\n\nContext:\n{context}\n\nProvide a concise answer."

    chat = client.chat.completions.create(
        model=LLM_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=temperature,
    )
    return chat.choices[0].message.content

# 3) Example usage
user_query = "Я хочу стать инжереном машиннего обучения. Какие курсы мне стоит взять из вашей платформы, чтобы начать свой путь?"
print(answer_with_rag(user_query, k=TOP_K))


Чтобы стать инженером машинного обучения, я рекомендую вам начать с курса по аналитике данных. Этот курс поможет вам освоить базовые навыки анализа данных, которые являются основой для дальнейшего изучения машинного обучения. Вы узнаете о работе аналитика данных, получите практические навыки и познакомитесь с основными инструментами. 

После завершения курса по аналитике данных, вы можете перейти к более специализированным курсам по машинному обучению, если они доступны на платформе.
